In [19]:
import pandas as pd
import os
import spacy
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api
import nltk
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag, wordnet
import nltk


directorio_base = os.path.join(os.getcwd(), "..")

In [20]:
# Descargar recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
# Ejecutar una vez para obtener dependencias necesarias
%pip install scikit-learn gensim

In [36]:
import pandas as pd

# Cargar los archivos CSV
movies_df = pd.read_csv('rotten_tomatoes_movies.csv')  # Archivo con información de películas
reviews_df = pd.read_csv('rotten_tomatoes_critic_reviews.csv')  # Archivo con críticas

# Seleccionar solo las columnas necesarias de cada archivo
movies_df = movies_df[['rotten_tomatoes_link', 'movie_title', 'movie_info', 'genres', 'tomatometer_status']]
reviews_df = reviews_df[['rotten_tomatoes_link', 'review_content']]

# Realizar la unión de los DataFrames usando la columna 'rotten_tomatoes_link'
merged_df = pd.merge(movies_df, reviews_df, on='rotten_tomatoes_link', how='inner')

# Asegurarse de que las críticas sean cadenas de texto, reemplazando los valores NaN por cadenas vacías
merged_df['review_content'] = merged_df['review_content'].fillna('')

# Agrupar las críticas por 'rotten_tomatoes_link' y concatenarlas en una sola celda
merged_df = merged_df.groupby(
    ['rotten_tomatoes_link', 'movie_title', 'movie_info', 'genres', 'tomatometer_status'],
    as_index=False
).agg({'review_content': lambda x: ', '.join(x)})

# Guardar el resultado en un nuevo archivo CSV
merged_df.to_csv('merged_movies_reviews.csv', index=False)

# Mostrar las primeras filas del DataFrame resultante
print(merged_df.head())



                    rotten_tomatoes_link  \
0                              m/0814255   
1                              m/0878835   
2                                   m/10   
3                 m/1000013-12_angry_men   
4  m/1000079-20000_leagues_under_the_sea   

                                         movie_title  \
0  Percy Jackson & the Olympians: The Lightning T...   
1                                        Please Give   
2                                                 10   
3                    12 Angry Men (Twelve Angry Men)   
4                       20,000 Leagues Under The Sea   

                                          movie_info  \
0  Always trouble-prone, the life of teenager Per...   
1  Kate (Catherine Keener) and her husband Alex (...   
2  A successful, middle-aged Hollywood songwriter...   
3  Following the closing arguments in a murder tr...   
4  In 1866, Professor Pierre M. Aronnax (Paul Luk...   

                                              genres tomatome

In [32]:
import pandas as pd
import re
from nltk.corpus import stopwords

# Configuración
stopwords_set = set(stopwords.words('english'))
signos_puntuacion_regex = re.compile(r"[,.:\-_\+\;\(\)\[\]%\$#@&\?!\/><]+")

# Preprocesamiento simplificado de 'review_content'
def preprocesar_review_content_rapido(contenido):
    contenido = contenido.lower()  # Convertir a minúsculas
    contenido = re.sub(r'\d+', '', contenido)  # Eliminar números
    contenido = signos_puntuacion_regex.sub('', contenido)  # Eliminar signos de puntuación
    contenido = re.sub(r'\s+', ' ', contenido)  # Espacios múltiples
    tokens = contenido.split()  # Tokenización básica
    return " ".join(word for word in tokens if word not in stopwords_set)  # Eliminar stopwords

# Cargar solo 'review_content'
df = pd.read_csv('merged_movies_reviews.csv', usecols=['review_content'])
df['review_content'] = df['review_content'].fillna('')  # Rellenar nulos

# Aplicar preprocesamiento rápido
df['review_content'] = df['review_content'].apply(preprocesar_review_content_rapido)

# Guardar resultados
df.to_csv('processed_review_content.csv', index=False)
print("Preprocesamiento completado. Guardado en 'processed1_review_content.csv'")


Preprocesamiento rápido completado. Guardado en 'processed1_review_content.csv'


In [38]:


# Cargar ambos archivos CSV
merged_df = pd.read_csv('merged_movies_reviews.csv')
processed_df = pd.read_csv('processed_review_content.csv')

# Asegurarse de que ambos DataFrames tengan la misma columna 'rotten_tomatoes_link' para realizar la unión
# Realizar la unión de los DataFrames usando 'rotten_tomatoes_link'
merged_df = pd.merge(merged_df, processed_df[['review_content']], left_index=True, right_index=True, how='inner')

# Sobrescribir la columna 'review_content' en merged_df con la de processed_df
merged_df['review_content'] = merged_df['review_content_y']

# Eliminar la columna auxiliar 'review_content_y'
merged_df.drop(columns=['review_content_y'], inplace=True)

# Guardar el DataFrame resultante en un nuevo archivo CSV
merged_df.to_csv('merged_and_processed_movies_reviews.csv', index=False)

# Mostrar las primeras filas del DataFrame resultante
print(merged_df.head())


                    rotten_tomatoes_link  \
0                              m/0814255   
1                              m/0878835   
2                                   m/10   
3                 m/1000013-12_angry_men   
4  m/1000079-20000_leagues_under_the_sea   

                                         movie_title  \
0  Percy Jackson & the Olympians: The Lightning T...   
1                                        Please Give   
2                                                 10   
3                    12 Angry Men (Twelve Angry Men)   
4                       20,000 Leagues Under The Sea   

                                          movie_info  \
0  Always trouble-prone, the life of teenager Per...   
1  Kate (Catherine Keener) and her husband Alex (...   
2  A successful, middle-aged Hollywood songwriter...   
3  Following the closing arguments in a murder tr...   
4  In 1866, Professor Pierre M. Aronnax (Paul Luk...   

                                              genres tomatome

In [41]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Cargar el archivo CSV preprocesado
merged_df = pd.read_csv('merged_and_processed_movies_reviews.csv')

# Asegurarse de que no haya valores NaN en 'review_content'
merged_df['review_content'] = merged_df['review_content'].fillna('')

# Vectorización con Bag of Words (BoW)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(merged_df['review_content'])  # Vectorizamos las críticas

# Función para responder consultas
def buscar_peliculas(query, vectorizer, corpus, top_n=5):
    """
    Responde a una consulta textual, devolviendo las películas más relevantes,
    incluyendo título, información y género.

    :param query: La consulta textual de ejemplo (str).
    :param vectorizer: El vectorizador BoW entrenado.
    :param corpus: El corpus de textos (críticas).
    :param top_n: Número de películas más relevantes a devolver.
    :return: DataFrame con las películas más relevantes.
    """
    # Convertir la consulta en un vector usando el vectorizador
    query_vector = vectorizer.transform([query])

    # Calcular la similitud coseno entre la consulta y el corpus
    similitudes = cosine_similarity(query_vector, corpus)

    # Obtener los índices de las top_n películas más similares
    indices_relevantes = similitudes.argsort()[0][-top_n:][::-1]

    # Obtener los detalles de las películas correspondientes a esos índices
    peliculas_relevantes = merged_df.iloc[indices_relevantes][['movie_title', 'movie_info', 'genres']]

    return peliculas_relevantes

# Ejemplo de consulta
consulta_1 = "Películas sobre viajes espaciales."
consulta_2 = "Películas para ver en familia."

# Buscar películas relevantes para ambas consultas
resultados_1 = buscar_peliculas(consulta_1, vectorizer, X)
resultados_2 = buscar_peliculas(consulta_2, vectorizer, X)

# Mostrar los resultados
print("Resultados para la consulta:", consulta_1)
print(resultados_1)
print("\nResultados para la consulta:", consulta_2)
print(resultados_2)


Resultados para la consulta: Películas sobre viajes espaciales.
                                 movie_title  \
16323                 Un Oso rojo (Red Bear)   
11415                  Paris When It Sizzles   
8348   Is Paris Burning? (Paris brûle-t-il?)   
15975                                Tiptoes   
1886                                    1984   

                                              movie_info  \
16323  Released from prison, a man (Julio Chávez) re-...   
11415  Hotshot Hollywood screenwriter Richard Benson ...   
8348   Near the end of World War II, Gen. Dietrich vo...   
15975  A man (Matthew McConaughey) is reluctant to te...   
1886   A man loses his identity while living under a ...   

                                           genres  
16323            Art House & International, Drama  
11415                   Classics, Comedy, Romance  
8348   Art House & International, Classics, Drama  
15975    Art House & International, Comedy, Drama  
1886    Drama, Science Fic